In [1]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display

from atj_trading.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [13]:
symbol = 'US500'
start_dt = datetime(2010, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)
ohlc['year'] = ohlc['time'].dt.year
ohlc['month'] = ohlc['time'].dt.month

display(ohlc)

fig = create_price_fig(ohlc)
display(fig)

,time,open,high,low,close,year,month
0,2012-08-06,1397.90,1398.90,1397.70,1398.40,2012,8
1,2012-08-07,1393.20,1407.20,1391.40,1400.20,2012,8
2,2012-08-08,1400.40,1403.40,1393.90,1400.20,2012,8
3,2012-08-09,1399.90,1406.70,1396.70,1403.00,2012,8
4,2012-08-10,1403.40,1403.40,1398.40,1399.20,2012,8
...,...,...,...,...,...,...,...
3099,2024-08-05,5298.96,5314.58,5089.71,5218.29,2024,8
3100,2024-08-06,5220.82,5312.51,5193.55,5219.38,2024,8
3101,2024-08-07,5213.59,5331.13,5170.15,5174.65,2024,8
3102,2024-08-08,5180.77,5333.86,5155.66,5326.44,2024,8


In [15]:
ohlc_by_month = ohlc.groupby(['year', 'month']).first()
days_to_buy = ohlc_by_month['time'].tolist()

days_to_buy

[Timestamp('2012-08-06 00:00:00'),
 Timestamp('2012-09-03 00:00:00'),
 Timestamp('2012-10-01 00:00:00'),
 Timestamp('2012-11-01 00:00:00'),
 Timestamp('2012-12-03 00:00:00'),
 Timestamp('2013-01-02 00:00:00'),
 Timestamp('2013-02-01 00:00:00'),
 Timestamp('2013-03-01 00:00:00'),
 Timestamp('2013-04-01 00:00:00'),
 Timestamp('2013-05-01 00:00:00'),
 Timestamp('2013-06-03 00:00:00'),
 Timestamp('2013-07-01 00:00:00'),
 Timestamp('2013-08-01 00:00:00'),
 Timestamp('2013-09-02 00:00:00'),
 Timestamp('2013-10-01 00:00:00'),
 Timestamp('2013-11-01 00:00:00'),
 Timestamp('2013-12-02 00:00:00'),
 Timestamp('2014-01-02 00:00:00'),
 Timestamp('2014-02-03 00:00:00'),
 Timestamp('2014-03-03 00:00:00'),
 Timestamp('2014-04-01 00:00:00'),
 Timestamp('2014-05-01 00:00:00'),
 Timestamp('2014-06-02 00:00:00'),
 Timestamp('2014-07-01 00:00:00'),
 Timestamp('2014-08-01 00:00:00'),
 Timestamp('2014-09-01 00:00:00'),
 Timestamp('2014-10-01 00:00:00'),
 Timestamp('2014-11-03 00:00:00'),
 Timestamp('2014-12-

In [16]:
def get_signal(x):
    if x['time'] in days_to_buy:
        return 'buy'
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc

,time,open,high,low,close,year,month,signal
0,2012-08-06,1397.90,1398.90,1397.70,1398.40,2012,8,buy
1,2012-08-07,1393.20,1407.20,1391.40,1400.20,2012,8,None
2,2012-08-08,1400.40,1403.40,1393.90,1400.20,2012,8,None
3,2012-08-09,1399.90,1406.70,1396.70,1403.00,2012,8,None
4,2012-08-10,1403.40,1403.40,1398.40,1399.20,2012,8,None
...,...,...,...,...,...,...,...,...
3099,2024-08-05,5298.96,5314.58,5089.71,5218.29,2024,8,None
3100,2024-08-06,5220.82,5312.51,5193.55,5219.38,2024,8,None
3101,2024-08-07,5213.59,5331.13,5170.15,5174.65,2024,8,None
3102,2024-08-08,5180.77,5333.86,5155.66,5326.44,2024,8,None


In [17]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 200 / data['open'] # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['signal'] == 'buy':
        orders.open_trade(symbol, volume, 'buy')
 

In [18]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -1

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative
0,closed,US500,buy,0.143072,2012-08-06 00:00:00,1397.9,2024-08-09 00:00:00,5337.06,0,0,563.58,-0.143072,563.436928,563.436928
1,closed,US500,buy,0.142258,2012-09-03 00:00:00,1405.9,2024-08-09 00:00:00,5337.06,0,0,559.24,-0.142258,559.097742,1122.534671
2,closed,US500,buy,0.139237,2012-10-01 00:00:00,1436.4,2024-08-09 00:00:00,5337.06,0,0,543.12,-0.139237,542.980763,1665.515434
3,closed,US500,buy,0.142126,2012-11-01 00:00:00,1407.2,2024-08-09 00:00:00,5337.06,0,0,558.54,-0.142126,558.397874,2223.913307
4,closed,US500,buy,0.141373,2012-12-03 00:00:00,1414.7,2024-08-09 00:00:00,5337.06,0,0,554.51,-0.141373,554.368627,2778.281935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,closed,US500,buy,0.038048,2024-04-01 00:00:00,5256.58,2024-08-09 00:00:00,5337.06,0,0,3.06,-0.038048,3.021952,31076.67131
141,closed,US500,buy,0.03984,2024-05-01 00:00:00,5020.08,2024-08-09 00:00:00,5337.06,0,0,12.63,-0.03984,12.59016,31089.26147
142,closed,US500,buy,0.03784,2024-06-03 00:00:00,5285.46,2024-08-09 00:00:00,5337.06,0,0,1.95,-0.03784,1.91216,31091.17363
143,closed,US500,buy,0.036542,2024-07-01 00:00:00,5473.08,2024-08-09 00:00:00,5337.06,0,0,-4.97,-0.036542,-5.006542,31086.167087


In [19]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [21]:
backtest_fig = bt.visualize_backtest()
backtest_fig

In [22]:
evaluate_backtest(bt.trades)

biggest_profit 563.58
biggest_loss -7.25


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative
0,closed,US500,buy,0.143072,2012-08-06,1397.9,2024-08-09,5337.06,0,0,563.58,-0.143072,563.436928,563.436928
1,closed,US500,buy,0.142258,2012-09-03,1405.9,2024-08-09,5337.06,0,0,559.24,-0.142258,559.097742,1122.534671
2,closed,US500,buy,0.139237,2012-10-01,1436.4,2024-08-09,5337.06,0,0,543.12,-0.139237,542.980763,1665.515434
3,closed,US500,buy,0.142126,2012-11-01,1407.2,2024-08-09,5337.06,0,0,558.54,-0.142126,558.397874,2223.913307
4,closed,US500,buy,0.141373,2012-12-03,1414.7,2024-08-09,5337.06,0,0,554.51,-0.141373,554.368627,2778.281935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,closed,US500,buy,0.041221,2024-02-01,4851.91,2024-08-09,5337.06,0,0,20.00,-0.041221,19.958779,31064.058636
139,closed,US500,buy,0.039278,2024-03-01,5091.85,2024-08-09,5337.06,0,0,9.63,-0.039278,9.590722,31073.649357
140,closed,US500,buy,0.038048,2024-04-01,5256.58,2024-08-09,5337.06,0,0,3.06,-0.038048,3.021952,31076.67131
141,closed,US500,buy,0.03984,2024-05-01,5020.08,2024-08-09,5337.06,0,0,12.63,-0.03984,12.59016,31089.26147


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative
143,closed,US500,buy,0.036542,2024-07-01,5473.08,2024-08-09,5337.06,0,0,-4.97,-0.036542,-5.006542,31086.167087
144,closed,US500,buy,0.036115,2024-08-01,5537.94,2024-08-09,5337.06,0,0,-7.25,-0.036115,-7.286115,31078.880973


avg_win 217.49902097902097
avg_loss -6.109999999999999
count_profit_trades 143
count_loss_trades 2
win_rate 71.5
rrr 35.6


,order_type,profit
0,buy,31090.14


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative,dayofweek
0,closed,US500,buy,0.143072,2012-08-06,1397.9,2024-08-09,5337.06,0,0,563.58,-0.143072,563.436928,563.436928,4
1,closed,US500,buy,0.142258,2012-09-03,1405.9,2024-08-09,5337.06,0,0,559.24,-0.142258,559.097742,1122.534671,4
2,closed,US500,buy,0.139237,2012-10-01,1436.4,2024-08-09,5337.06,0,0,543.12,-0.139237,542.980763,1665.515434,4
3,closed,US500,buy,0.142126,2012-11-01,1407.2,2024-08-09,5337.06,0,0,558.54,-0.142126,558.397874,2223.913307,4
4,closed,US500,buy,0.141373,2012-12-03,1414.7,2024-08-09,5337.06,0,0,554.51,-0.141373,554.368627,2778.281935,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,closed,US500,buy,0.038048,2024-04-01,5256.58,2024-08-09,5337.06,0,0,3.06,-0.038048,3.021952,31076.67131,4
141,closed,US500,buy,0.03984,2024-05-01,5020.08,2024-08-09,5337.06,0,0,12.63,-0.03984,12.59016,31089.26147,4
142,closed,US500,buy,0.03784,2024-06-03,5285.46,2024-08-09,5337.06,0,0,1.95,-0.03784,1.91216,31091.17363,4
143,closed,US500,buy,0.036542,2024-07-01,5473.08,2024-08-09,5337.06,0,0,-4.97,-0.036542,-5.006542,31086.167087,4


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative,dayofweek,current_max,drawdown
0,closed,US500,buy,0.143072,2012-08-06,1397.9,2024-08-09,5337.06,0,0,563.58,-0.143072,563.436928,563.436928,4,563.436928,0.0
1,closed,US500,buy,0.142258,2012-09-03,1405.9,2024-08-09,5337.06,0,0,559.24,-0.142258,559.097742,1122.534671,4,1122.534671,0.0
2,closed,US500,buy,0.139237,2012-10-01,1436.4,2024-08-09,5337.06,0,0,543.12,-0.139237,542.980763,1665.515434,4,1665.515434,0.0
3,closed,US500,buy,0.142126,2012-11-01,1407.2,2024-08-09,5337.06,0,0,558.54,-0.142126,558.397874,2223.913307,4,2223.913307,0.0
4,closed,US500,buy,0.141373,2012-12-03,1414.7,2024-08-09,5337.06,0,0,554.51,-0.141373,554.368627,2778.281935,4,2778.281935,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,closed,US500,buy,0.038048,2024-04-01,5256.58,2024-08-09,5337.06,0,0,3.06,-0.038048,3.021952,31076.67131,4,31076.671310,0.0
141,closed,US500,buy,0.03984,2024-05-01,5020.08,2024-08-09,5337.06,0,0,12.63,-0.03984,12.59016,31089.26147,4,31089.261470,0.0
142,closed,US500,buy,0.03784,2024-06-03,5285.46,2024-08-09,5337.06,0,0,1.95,-0.03784,1.91216,31091.17363,4,31091.173630,0.0
143,closed,US500,buy,0.036542,2024-07-01,5473.08,2024-08-09,5337.06,0,0,-4.97,-0.036542,-5.006542,31086.167087,4,31091.173630,-5.006542


ValueError: Plotly Express cannot process wide-form data with columns of different type.